In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# import X_test and y_test
X_test = pd.read_csv('Datasets\VanilaDataset\X-Y Splitted Data\X_test.csv')
y_test = pd.read_csv('Datasets\VanilaDataset\X-Y Splitted Data\y_test.csv')

# 1st column in these dataets are the level 1 index , and the 2nd column is the level 2 index , set them as such
X_test = X_test.set_index(['Instance', 'Time'])  
y_test = y_test.set_index(['Instance'])  

<>:2: SyntaxWarning: invalid escape sequence '\V'
<>:3: SyntaxWarning: invalid escape sequence '\V'
<>:2: SyntaxWarning: invalid escape sequence '\V'
<>:3: SyntaxWarning: invalid escape sequence '\V'
C:\Users\gwiku\AppData\Local\Temp\ipykernel_32816\2234204653.py:2: SyntaxWarning: invalid escape sequence '\V'
  X_test = pd.read_csv('Datasets\VanilaDataset\X-Y Splitted Data\X_test.csv')
C:\Users\gwiku\AppData\Local\Temp\ipykernel_32816\2234204653.py:3: SyntaxWarning: invalid escape sequence '\V'
  y_test = pd.read_csv('Datasets\VanilaDataset\X-Y Splitted Data\y_test.csv')


In [7]:
X_test

Open      High       Low     Close    Volume
Instance Time                                                  
0        0     0.260947  0.501340  0.170688  0.498660  0.416074
         1     0.353888  0.401252  0.015192  0.140304  1.000000
         2     0.176050  0.317248  0.000000  0.195711  0.304512
         3     0.195711  0.757820  0.166220  0.697051  0.886552
         4     0.859697  0.974085  0.579982  0.609473  0.317423
...                 ...       ...       ...       ...       ...
538      5     0.227648  0.295267  0.000000  0.273479  0.021680
         6     0.250939  0.500375  0.172802  0.428249  0.048380
         7     0.477085  0.660406  0.262960  0.441022  0.288909
         8     0.569497  1.000000  0.555222  0.965440  0.456641
         9     0.981217  0.981217  0.643877  0.765590  0.261296

[14492 rows x 5 columns]

In [8]:
y_test

,Pattern
Instance,
0,4
1,0
2,1
3,4
4,1
...,...
534,2
535,6
536,0


In [9]:
# max min and avg lengths of Instance index sets in X_test
max_len = 0
min_len = 100000
sum_len = 0
for i in range(1, 538):
    len_i = len(X_test.loc[i])
    sum_len += len_i
    if len_i > max_len:
        max_len = len_i
    if len_i < min_len:
        min_len = len_i
        
avg_len = sum_len / 537
max_len, min_len, avg_len

(275, 5, 26.949720670391063)

In [32]:
#  load test_patterns_with_symbols.csv and train_patterns_with_symbols.csv
test_patterns = pd.read_csv('Datasets/VanilaDataset/test_patterns_with_symbols.csv')
train_patterns = pd.read_csv('Datasets/VanilaDataset/train_patterns_with_symbols.csv')

# convert the Start and End columns to datetime
test_patterns['Start'] = pd.to_datetime(test_patterns['Start'])
test_patterns['End'] = pd.to_datetime(test_patterns['End'])
train_patterns['Start'] = pd.to_datetime(train_patterns['Start'])
train_patterns['End'] = pd.to_datetime(train_patterns['End'])


In [40]:
test_patterns

,Symbol,Chart Pattern,BullishBearish,Start,End,Industry
0,ADSK,"Double Bottom, Adam and Adam",0,2023-09-27,2023-10-05,Computer Software and Svcs
1,CL,"Double Top, Adam and Adam",0,2023-05-04,2023-05-12,Household Products
2,ADBE,"Triangle, symmetrical",1,2020-07-13,2020-07-30,Computer Software and Svcs
4,FISV,"Double Bottom, Eve and Adam",1,2020-12-23,2021-01-04,IT Services
5,FLS,"Triangle, symmetrical",1,2022-01-18,2022-02-07,Machinery
...,...,...,...,...,...,...
536,TSCO,"Double Bottom, Eve and Adam",0,2023-07-17,2023-07-25,Retail Building Supply
537,NVDA,"Flag, high and tight",0,2024-01-04,2024-03-07,Semiconductor
538,FTV,"Double Top, Adam and Adam",0,2024-09-27,2024-10-15,Industrial Services
539,DOX,"Double Top, Adam and Adam",0,2022-12-02,2022-12-14,IT Services


In [38]:
test_patterns.loc[3, 'Symbol'] == train_patterns.loc[3, 'Symbol']

False

In [39]:
for i in range(len(test_patterns)):
    for j in range(len(train_patterns)):
        if (
            test_patterns.loc[i, 'Start'] < train_patterns.loc[j, 'End'] or
            test_patterns.loc[i, 'End'] > train_patterns.loc[j, 'Start'] and
            test_patterns.loc[i, 'Symbol'] == train_patterns.loc[j, 'Symbol']
        ):
            print('test start and end ', test_patterns.loc[i, 'Start'], test_patterns.loc[i, 'End'])
            print('train start and end ', train_patterns.loc[j, 'Start'], train_patterns.loc[j, 'End'])
            test_patterns = test_patterns.drop(i)
            break


test start and end  2023-05-16 00:00:00 2023-05-31 00:00:00
train start and end  2023-02-14 00:00:00 2023-08-09 00:00:00
test start and end  2024-08-07 00:00:00 2024-08-12 00:00:00
train start and end  2024-08-01 00:00:00 2024-09-19 00:00:00
test start and end  2024-07-18 00:00:00 2024-07-31 00:00:00
train start and end  2024-07-18 00:00:00 2024-07-31 00:00:00
test start and end  2024-09-03 00:00:00 2024-09-12 00:00:00
train start and end  2024-09-03 00:00:00 2024-09-12 00:00:00
test start and end  2023-08-03 00:00:00 2023-08-18 00:00:00
train start and end  2023-07-27 00:00:00 2023-10-12 00:00:00
test start and end  2022-05-11 00:00:00 2022-07-07 00:00:00
train start and end  2022-03-29 00:00:00 2022-07-21 00:00:00
test start and end  2023-09-01 00:00:00 2023-09-28 00:00:00
train start and end  2023-09-11 00:00:00 2023-09-21 00:00:00
test start and end  2023-10-23 00:00:00 2023-10-27 00:00:00
train start and end  2023-07-19 00:00:00 2023-12-04 00:00:00
test start and end  2020-03-17 0

In [ ]:
new_combined_test_ohcl_df = pd.DataFrame()

for index , row in X_test.iterrows():
    current_pattern_start_date = row['Start']
    current_pattern_end_date = row['End']
    
    # add 300 days to the current pattern end date and get the new pattern end date and go back 300 days to get the new pattern start date
    new_pattern_end_date = current_pattern_end_date + pd.DateOffset(days=300)
    new_pattern_start_date = current_pattern_start_date - pd.DateOffset(days=300)
    
    for j in range(len(train_patterns)):
        if (
                (new_pattern_start_date < train_patterns.loc[j, 'Start'] and
                new_pattern_end_date > train_patterns.loc[j, 'Start']) and
                row ['Symbol'] == train_patterns.loc[j, 'Symbol']
            ):
            
            new_pattern_end_date = train_patterns.loc[j, 'Start']
            
        elif (
                (new_pattern_start_date < train_patterns.loc[j, 'End'] and
                new_pattern_end_date > train_patterns.loc[j, 'End']) and
                row ['Symbol'] == train_patterns.loc[j, 'Symbol']
            ):
            
            new_pattern_start_date = train_patterns.loc[j, 'End']